In [3]:
import random
import string
import tkinter as tk
from tkinter import messagebox, ttk
import cx_Oracle

# Establish connection to the database
conn = cx_Oracle.connect('system/tiger@localhost:1521/xe')
cursor = conn.cursor()

# Check if the 'bank' table exists, and create it if not
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'CREATE TABLE bank (username VARCHAR2(50), password VARCHAR2(50), balance NUMBER, account_number NUMBER)';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -955 THEN
                RAISE;
            END IF;
    END;
""")
conn.commit()

class Bank:
    def __init__(self):
        self.current_user = None

    def is_strong_password(self, password):
        if len(password) < 9:
            return False
        if not any(char in string.punctuation for char in password):
            return False
        if not any(char.isupper() for char in password):
            return False
        if not any(char.islower() for char in password):
            return False
        return True

    def register_user(self, username, password):
        
        if username in self.get_usernames():
            return "Username already exists. Please choose another one."

        if not self.is_strong_password(password):
            return "Password is not strong enough. It should contain at least one special character, one capital letter, one small alphabet, and have a minimum length of 9 characters."

        account_number = random.randint(10000, 99999)

    # Store user information in the database
        cursor.execute("INSERT INTO bank (username, password, balance, account_number) VALUES (:1, :2, :3, :4)",
               (username, password, 0, account_number))


        conn.commit()

        return f"User {username} registered with account number {account_number}."

    def login(self, username, password):
        cursor.execute("SELECT * FROM bank WHERE username = :1 AND password = :2",
                       (username, password))
        result = cursor.fetchone()

        if result:
            self.current_user = username
            return f"Login successful for {username}."
        return "Invalid username or password."

    def logout(self):
        self.current_user = None
        return "Logged out successfully."

    def deposit(self, amount):
        if self.current_user:
            cursor.execute("UPDATE bank SET balance = balance + :1 WHERE username = :2",
                           (amount, self.current_user))
            conn.commit()
            return f"Deposited ${amount}. New balance: ${self.get_balance()}"
        return "Please log in first."

    def withdraw(self, amount):
        if self.current_user:
            if self.get_balance() >= amount:
                cursor.execute("UPDATE bank SET balance = balance - :1 WHERE username = :2",
                               (amount, self.current_user))
                conn.commit()
                return f"Withdrew ${amount}. New balance: ${self.get_balance()}"
            else:
                return "Insufficient balance."
        return "Please log in first."

    def check_balance(self):
        if self.current_user:
            return f"Balance for {self.current_user}: ${self.get_balance()}"
        return "Please log in first."

    def delete_account(self):
        if self.current_user:
            cursor.execute("DELETE FROM bank WHERE username = :1", (self.current_user,))
            conn.commit()
            self.current_user = None
            return f"Account deleted for {self.current_user}."

    def get_balance(self):
        cursor.execute("SELECT balance FROM bank WHERE username = :1", (self.current_user,))
        return cursor.fetchone()[0]

    def get_usernames(self):
        cursor.execute("SELECT username FROM bank")
        return [row[0] for row in cursor.fetchall()]

class BankGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ABC Bank")
        self.geometry("400x300")

        self.bank = Bank()

        self.frames = {}
        self.create_frames()
        self.show_frame(StartPage)

    def create_frames(self):
        for F in (StartPage, RegisterPage, LoginPage, BankingPage):
            frame = F(self, self.bank)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

class StartPage(tk.Frame):
    def __init__(self, master, bank):
        tk.Frame.__init__(self, master)
        self.bank = bank

        #label = tk.Label(self, text="Welcome to ABC Bank", font=("Helvetica", 16))
        label = tk.Label(self, text="         Welcome to MV Bank", font=("Helvetica", 20))
        label.pack(pady=10)

        register_button = tk.Button(self, text="Register", command=lambda: master.show_frame(RegisterPage))
        register_button.pack()

        login_button = tk.Button(self, text="Login", command=lambda: master.show_frame(LoginPage))
        login_button.pack()

        quit_button = tk.Button(self, text="Quit", command=master.destroy)
        quit_button.pack()

class RegisterPage(tk.Frame):
    def __init__(self, master, bank):
        tk.Frame.__init__(self, master)
        self.bank = bank

        label = tk.Label(self, text="Register", font=("Helvetica", 16))
        label.pack(pady=10)

        username_label = tk.Label(self, text="Enter a username:")
        username_label.pack()

        self.username_entry = tk.Entry(self)
        self.username_entry.pack(pady=10)

        password_label = tk.Label(self, text="Enter a password:")
        password_label.pack()

        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.pack(pady=10)

        register_button = tk.Button(self, text="Register", command=self.register_user)
        register_button.pack()

        back_button = tk.Button(self, text="Back to Home", command=lambda: master.show_frame(StartPage))
        back_button.pack()

    def register_user(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        result = self.bank.register_user(username, password)
        messagebox.showinfo("Registration Result", result)

class LoginPage(tk.Frame):
    def __init__(self, master, bank):
        tk.Frame.__init__(self, master)
        self.bank = bank

        label = tk.Label(self, text="Login", font=("Helvetica", 16))
        label.pack(pady=10)

        username_label = tk.Label(self, text="Enter your username:")
        username_label.pack()

        self.username_entry = tk.Entry(self)
        self.username_entry.pack(pady=10)

        password_label = tk.Label(self, text="Enter your password:")
        password_label.pack()

        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.pack(pady=10)

        login_button = tk.Button(self, text="Login", command=self.login)
        login_button.pack()

        back_button = tk.Button(self, text="Back to Home", command=lambda: self.master.show_frame(StartPage))
        back_button.pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        result = self.bank.login(username, password)
        messagebox.showinfo("Login Result", result)

        if "successful" in result.lower():
            self.username_entry.delete(0, tk.END)
            self.password_entry.delete(0, tk.END)
            self.master.show_frame(BankingPage)


class BankingPage(tk.Frame):
    def __init__(self, master, bank):
        tk.Frame.__init__(self, master)
        self.bank = bank

        label = tk.Label(self, text="Banking Operations", font=("Helvetica", 16))
        label.pack(pady=10)

        deposit_button = tk.Button(self, text="Deposit", command=self.deposit)
        deposit_button.pack(pady=5)

        withdraw_button = tk.Button(self, text="Withdraw", command=self.withdraw)
        withdraw_button.pack(pady=5)

        check_balance_button = tk.Button(self, text="Check Balance", command=self.check_balance)
        check_balance_button.pack(pady=5)

        delete_account_button = tk.Button(self, text="Delete Account", command=self.delete_account)
        delete_account_button.pack(pady=5)

        logout_button = tk.Button(self, text="Logout", command=self.logout)
        logout_button.pack(pady=5)

    def deposit(self):
        deposit_window = DepositWindow(self.master, self.bank, self)
        deposit_window.grab_set()

    def withdraw(self):
        withdraw_window = WithdrawWindow(self.master, self.bank, self)
        withdraw_window.grab_set()

    def check_balance(self):
        result = self.bank.check_balance()
        messagebox.showinfo("Balance Result", result)

    def delete_account(self):
        result = self.bank.delete_account()
        messagebox.showinfo("Delete Account Result", result)
        self.master.show_frame(StartPage)

    def logout(self):
        result = self.bank.logout()
        messagebox.showinfo("Logout Result", result)
        self.master.show_frame(StartPage)

class DepositWindow(tk.Toplevel):
    def __init__(self, master, bank, parent_frame):
        tk.Toplevel.__init__(self, master)
        self.bank = bank
        self.parent_frame = parent_frame

        self.title("Deposit")
        self.geometry("200x150")

        label = tk.Label(self, text="Enter amount to deposit:")
        label.pack(pady=10)

        self.amount_entry = tk.Entry(self)
        self.amount_entry.pack(pady=5)

        deposit_button = tk.Button(self, text="Deposit", command=self.deposit)
        deposit_button.pack(pady=5)

    def deposit(self):
        try:
            amount = float(self.amount_entry.get())
            result = self.bank.deposit(amount)
            messagebox.showinfo("Deposit Result", result)
            self.parent_frame.master.show_frame(BankingPage)
            self.destroy()
        except ValueError:
            messagebox.showerror("Error", "Invalid input. Please enter a valid number.")

class WithdrawWindow(tk.Toplevel):
    def __init__(self, master, bank, parent_frame):
        tk.Toplevel.__init__(self, master)
        self.bank = bank
        self.parent_frame = parent_frame

        self.title("Withdraw")
        self.geometry("200x150")

        label = tk.Label(self, text="Enter amount to withdraw:")
        label.pack(pady=10)

        self.amount_entry = tk.Entry(self)
        self.amount_entry.pack(pady=5)

        withdraw_button = tk.Button(self, text="Withdraw", command=self.withdraw)
        withdraw_button.pack(pady=5)

    def withdraw(self):
        try:
            amount = float(self.amount_entry.get())
            result = self.bank.withdraw(amount)
            messagebox.showinfo("Withdraw Result", result)
            self.parent_frame.master.show_frame(BankingPage)
            self.destroy()
        except ValueError:
            messagebox.showerror("Error", "Invalid input. Please enter a valid number.")

if __name__ == "__main__":
    app = BankGUI()
    app.mainloop()
